# Run Theta Model
Steps:
1. Open dataset file
2. Run Theta model (including save to file)
3. Evaluation

# 1. Open Dataset File

## Import libraries

In [200]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd 
import collections
import matplotlib.pyplot as plt

# Import exponential smoothing model
from statsmodels.tsa.holtwinters import ExponentialSmoothing 

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings("ignore")

## Read dataset files

In [201]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [202]:
# Read X & Y datasets from Pickle file, along with the metadata
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset.pickle", "rb") as f:
    X = pickle.load(f)
    Y = pickle.load(f)  # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# 2. Run Theta model

## Create functions

In [203]:
# Function to calculate 5% band i.e. 
#     0 to 5% = U1 (including 0), 5 to 10% = U2, 10 to 15% = U3, 15 to 20% = U4, >= U5.
#     <0 to -5% = D1, -5 to -10% = D2, -10 to -15% = D3, -15 to -20 = D4, <-20% = D5.
def calc_band(input_pct):
    if input_pct > 20: 
        band = 5
        prefix = 'U'
    elif input_pct < -20: 
        band = 5
        prefix = "D"
    elif input_pct >= 0:
        band = math.floor(input_pct / 5) + 1 # Floor(1/5) is 0 so to make it Band 1 we need to add 1
        prefix = "U"
    elif input_pct < 0:
        band = math.floor(input_pct / 5) # Floor(-1/5) is -1 so no need to add 1
        prefix = "D"
    else: # NaN for example
        band = 0
        prefix = "E" # Error band
    return prefix + str(abs(band))

## Calculate Theta Lines for 1000 datasets

In [204]:
# Set the coefficients for each Theta line
Theta1_Coef = 0.25  # The coefficient for Theta1 line
Theta2_Coef = 0.5 # The coefficient for Theta2 line
Theta3_Coef = 1.25  # The coefficient for Theta3 line
Theta4_Coef = 1.5    # The coefficient for Theta4 line

In [205]:
# Function to calculate "Theta 1" line with Theta coefficient = 0.5
def Calculate_Theta_Line (Theta_Coef, Theta_Column):
    Theta_Diff2 = Diff2_Today * Theta_Coef
    Theta_Diff1 = Theta_Diff2 + Diff1_Today
    Theta_Price = Theta_Diff1 + Price_Prev
    X[DS][Theta_Column].iloc[day-2] = Theta_Price
    return 0

In [206]:
# Define the number of samples once
Number_of_Samples = 1000

In [207]:
# Calculate Theta Lines for 1000 datasets with 4 Theta lines
# Run time: 1 minute for 22 datasets, about 45 mins for 1000 datasets (30 mins for 2 Theta lines)
for DS in range(Number_of_Samples): 

    # Add columns to calculate Theta lines
    X[DS]["Diff1"] = np.nan
    X[DS]["Diff2"] = np.nan
    X[DS]["Theta1"] = np.nan
    X[DS]["Theta2"] = np.nan
    X[DS]["Theta3"] = np.nan
    X[DS]["Theta4"] = np.nan

    # Loop for all days in the dataset
    for day in range(1, len(X[DS])): 

        # Calculate the first level difference (starting from day 1, not day 0)
        Price_Today = X[DS]["HL_Avg"].iloc[day]
        Price_Prev  = X[DS]["HL_Avg"].iloc[day-1]
        Diff1_Today = Price_Prev - Price_Today
        X[DS]["Diff1"].iloc[day] = Diff1_Today

        # Calculate the second level difference (starting from day 2, not day 1)
        if day > 1:
            Price_Prev2 = X[DS]["HL_Avg"].iloc[day-2]
            Diff1_Prev  = Price_Prev2 - Price_Prev
            Diff2_Today = Diff1_Prev - Diff1_Today
            X[DS]["Diff2"].iloc[day] = Diff2_Today

            # Calculate Theta Lines with different Theta coefficients
            Calculate_Theta_Line(Theta1_Coef, "Theta1")
            Calculate_Theta_Line(Theta2_Coef, "Theta2")
            Calculate_Theta_Line(Theta3_Coef, "Theta3")
            Calculate_Theta_Line(Theta4_Coef, "Theta4")

    # Monitor the run of 1000 dataset
    print("Dataset:", DS)

Dataset: 0
Dataset: 1
Dataset: 2
Dataset: 3
Dataset: 4
Dataset: 5
Dataset: 6
Dataset: 7
Dataset: 8
Dataset: 9
Dataset: 10
Dataset: 11
Dataset: 12
Dataset: 13
Dataset: 14
Dataset: 15
Dataset: 16
Dataset: 17
Dataset: 18
Dataset: 19
Dataset: 20
Dataset: 21
Dataset: 22
Dataset: 23
Dataset: 24
Dataset: 25
Dataset: 26
Dataset: 27
Dataset: 28
Dataset: 29
Dataset: 30
Dataset: 31
Dataset: 32
Dataset: 33
Dataset: 34
Dataset: 35
Dataset: 36
Dataset: 37
Dataset: 38
Dataset: 39
Dataset: 40
Dataset: 41
Dataset: 42
Dataset: 43
Dataset: 44
Dataset: 45
Dataset: 46
Dataset: 47
Dataset: 48
Dataset: 49
Dataset: 50
Dataset: 51
Dataset: 52
Dataset: 53
Dataset: 54
Dataset: 55
Dataset: 56
Dataset: 57
Dataset: 58
Dataset: 59
Dataset: 60
Dataset: 61
Dataset: 62
Dataset: 63
Dataset: 64
Dataset: 65
Dataset: 66
Dataset: 67
Dataset: 68
Dataset: 69
Dataset: 70
Dataset: 71
Dataset: 72
Dataset: 73
Dataset: 74
Dataset: 75
Dataset: 76
Dataset: 77
Dataset: 78
Dataset: 79
Dataset: 80
Dataset: 81
Dataset: 82
Dataset: 83
Da

# Forecast Theta lines

## Create forecast function

In [208]:
# Create a function to forecast price band using Double ES Additive model (which was the best ES model)

# Parameters: 
#   - Number of Samples (int): number of loop, i.e. 100 (default) or 1000
#   - Theta_Column = name of the Theta column to use

# Returns Result DF: Result data frame containing the forecasted bands

def Forecast_Using_ES_Model (Number_of_Samples = 1, Theta_Column="Theta1"):
    F_Avg_List   = [] # List of actual band

    # Forecast the price band for the first 1000 datasets (the remaining 2989 datasets are for ML training)
    for ds in range(Number_of_Samples):
    
        # Set the date as the index 
        X_ES = X[ds][["Date",Theta_Column]].set_index("Date")
        
        # Remove rows with Theta column = NaN
        X_ES2 = X_ES[~X_ES[Theta_Column].isnull()]

        # Create an ES model
        fitted_model = ExponentialSmoothing(X_ES2, trend = "add").fit()
    
        # Y variable consists of: Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
        Forecast_From = len(X_ES2) + int(Y[ds][4]) # Y[4] is Y_Gap (number of days from X Last Price). Tipically 64.
        Forecast_To   = Forecast_From + int(Y[ds][5]) # Y[5] is Y_Days (number of days in a week for Y). Tipically 5.
        
        F = fitted_model.forecast(Forecast_To)

        # Calculate forecast price band
        F_Avg = F[Forecast_From:Forecast_To].mean() # Average the 1 week prices (tipically 65 to 69)
        F_Avg_List.append(F_Avg)

    # Return the forecasted bands
    return F_Avg_List

In [209]:
def Calc_F_Band(F_Theta):
    X_Last = float(Y[DS][1]) # Y(1) is the last price in X 5 year range
    F_Pct = (F_Theta - X_Last) / X_Last * 100
    F_Band = calc_band(F_Pct) # The forecast band
    return F_Band

In [210]:
F_Theta1 = Forecast_Using_ES_Model(Number_of_Samples,"Theta1")
F_Theta2 = Forecast_Using_ES_Model(Number_of_Samples,"Theta2")
F_Theta3 = Forecast_Using_ES_Model(Number_of_Samples,"Theta3")
F_Theta4 = Forecast_Using_ES_Model(Number_of_Samples,"Theta4")

In [211]:
Theta_Band_List = []
Output_File = ""
for DS in range(Number_of_Samples):
    # Calculate the band of the average of Theta lines
    Band_14   = Calc_F_Band((F_Theta1[DS] + F_Theta4[DS])/2)
    Band_23   = Calc_F_Band((F_Theta2[DS] + F_Theta3[DS])/2)
    Band_1234 = Calc_F_Band((F_Theta1[DS] + F_Theta2[DS] + F_Theta3[DS] + F_Theta4[DS])/4)
    Actual_Price = Y[DS][1]
    Actual_Band  = Y[DS][3]
    
    # Compare the Theta bands to the actual band
    if Actual_Band == Band_14: Status_14 = 1
    else: Status_14 = 0
    if Actual_Band == Band_23: Status_23 = 1
    else: Status_23 = 0
    if Actual_Band == Band_1234: Status_1234 = 1
    else: Status_1234 = 0

    # Put prices, bands and statuses into a list
    Theta_Band = [ DS, Actual_Price, F_Theta1[DS], F_Theta2[DS], F_Theta3[DS], F_Theta4[DS],
                   Actual_Band, Band_14, Band_23, Band_1234, Status_14, Status_23, Status_1234 ]
    Theta_Band_List.append(Theta_Band)

    # Put prices, bands and statuses into a string for writing into a CSV file
    Output_Row = str(DS) + "," + str(Actual_Price) + "," + str(F_Theta1[DS]) + "," + \
                 str(F_Theta2[DS]) + "," + str(F_Theta3[DS]) + "," + str(F_Theta4[DS]) + "," + \
                 Actual_Band + "," + Band_14 + "," + Band_23 + "," + Band_1234 + "," + \
                 str(Status_14) + "," + str(Status_23) + "," + str(Status_1234) + "\n"
    Output_File = Output_File + Output_Row
    #print(Output_Row)

In [212]:
Theta_DF = pd.DataFrame(Theta_Band_List, 
                             columns = ["Dataset", "Actual_Price", "F_Theta1", "F_Theta2", "F_Theta3", "F_Theta4",
                                        "Actual_Band", "Band_14", "Band_23", "Band_1234", 
                                        "Status_14", "Status_23", "Status_1234"])
Theta_DF

Dataset  Actual_Price    F_Theta1    F_Theta2    F_Theta3    F_Theta4  \
0          0      0.474330    0.558909    0.563511    0.572914   -1.170252   
1          1      0.379464    0.705037    0.706240    1.554810    1.633171   
2          2      0.413125    0.460882    0.478225    0.484414    0.473906   
3          3      0.332946    0.436812    0.439539    0.426970    0.426240   
4          4      0.781529    1.161860    1.166965    1.231074    1.241514   
..       ...           ...         ...         ...         ...         ...   
995      995     71.494999   86.039689   85.909250   84.714556   84.599299   
996      996    113.770001  131.535426  131.785259  131.249828  130.513259   
997      997     65.990002   76.147947   75.392332   74.706645   75.075305   
998      998     82.729999  104.989450  105.118745  104.598219  104.559561   
999      999     71.674999   77.416805   77.232453   77.388796   77.313119   

    Actual_Band Band_14 Band_23 Band_1234  Status_14  Status_23  Status_1234  
0            U1      D5      U4        D5          0          0            0  
1            D3      U5      U5        U5          0          0            0  
2            U1      U3      U4        U3          0          0            0  
3            U5      U5      U5        U5          1          1            1  
4            U5      U5      U5        U5          1          1            1  
..          ...     ...     ...       ...        ...        ...          ...  
995          U2      U4      U4        U4          0          0            0  
996          D2      U4      U4        U4          0          0            0  
997          U2      U3      U3        U3          0          0            0  
998          D3      U5      U5        U5          0          0            0  
999          U1      U2      U2        U2          0          0            0  

[1000 rows x 13 columns]

In [213]:
print(sum(Theta_DF["Status_14"]))
print(sum(Theta_DF["Status_23"]))
print(sum(Theta_DF["Status_1234"]))

157
153
156


In [214]:
File_Path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/"
f = open(File_Path + "Theta.csv", "w")
f.write(Output_File)
f.close()